# **Projeto: Pipeline de Dados de Usuários para Análise de Marketing e Produto**

Elaborado por: **Renan Lima**

Data: 08/02/2026

## **Introdução** 

### **Contextualização do Problema:**
Para uma empresa ter sucesso, ela precisa entender seus usuários, mas os dados brutos de uma API não são suficientes. É necessário um pipeline de Engenharia de Dados para extrair, processar e transformar esses dados em informações úteis, respondendo a perguntas críticas de negócio e permitindo que as equipes tomem ações estratégicas.

### **Objetivo do Projeto:** 
O objetivo central deste projeto é construir um pipeline de dados ETL (Extração, Transformação e Carga) robusto e automatizado. Este pipeline irá:

1. **Extrair** de forma completa todos os dados de usuários da API GoRest, lidando com a complexidade da paginação.
2. **Transformar** os dados brutos utilizando a biblioteca Pandas para atender a duas necessidades de negócio distintas e de alta prioridade.
3. **Carregar** (neste caso, gerar) dois produtos de dados finais, prontos para consumo:
    - Uma lista segmentada para uma campanha de reativação do time de Marketing.
    - Um resumo agregado para análise de perfil de usuário do time de Produto.

In [4]:
# Configurações iniciais.
import requests 
import os 
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
TOKEN = os.getenv("API_TOKEN")
HEADERS = {"Authorization": f"Bearer {TOKEN}"}
URL = "https://gorest.co.in/public/v2"
PARAMS = {
    "page": 1}

In [ ]:
# Extração e Transformação para Uses Cases. 
response = requests.get(f"{URL}/users", headers=HEADERS, params=PARAMS)
response.raise_for_status()

all_data = response.json()
total_pages = int(response.headers["x-pagination-pages"])

for page in range(2, total_pages + 1):
    PARAMS["page"] = page
    response = requests.get(f"{URL}/users", headers=HEADERS, params=PARAMS)
    response.raise_for_status()
    data = response.json()
    all_data.extend(data)

df = pd.DataFrame.from_dict(all_data, orient="columns")

### **Use Case 1: Reativando Usuários para o Time de Marketing***
**Necessidade de Negócio:** O time de Marketing precisa identificar usuários que não estão mais engajados com a plataforma (status "inativo") para incluí-los em uma campanha de e-mail marketing personalizada. O objetivo é reengajá-los, diminuindo a taxa de churn (abandono).

**Solução do Pipeline:**
- Extração: Todos os usuários são extraídos da API.
- Transformação: O pipeline filtra o DataFrame para manter apenas os usuários com status == 'inactive'. Em seguida, ele cria uma nova coluna First Name extraindo apenas o primeiro nome do usuário para permitir a personalização do e-mail (ex: "Olá, Renan!"). Por fim, seleciona apenas as colunas essenciais (First Name e email).
- Entrega: O resultado é salvo no arquivo contatos_marketing.csv, uma lista limpa e pronta para ser importada na ferramenta de e-mail marketing.

In [ ]:
# Use Case - Time de Marketing.
base_marketing = df[df['status'] == "inactive"].copy()
base_marketing["First Name"] = base_marketing['name'].str.split(expand=True)[0]
base_marketing = base_marketing[['First Name', "email"]]
base_marketing.to_csv("data/contatos_marketing.csv", index=False)

### **Use Case 2: Análise Demográfica para o Time de Produto**
**Necessidade de Negócio:** O time de Produto quer entender o perfil demográfico do seu público mais fiel (status "ativo") para guiar o desenvolvimento de novas funcionalidades. A primeira pergunta é sobre a distribuição de gênero, para validar se as funcionalidades atuais estão alinhadas com o público principal.

**Solução do Pipeline:**

- Extração: Os mesmos dados extraídos da API são aproveitados.
- Transformação: O pipeline filtra o DataFrame para manter apenas os usuários com status == 'active'. Em seguida, ele aplica uma contagem de valores (value_counts()) na coluna gender para agregar os dados.
- Entrega: O resultado é um resumo numérico (ex: Male: 488, Female: 546) que responde diretamente à pergunta do time de Produto, permitindo uma análise rápida. Este resultado é salvo no arquivo relatorio_genero_ativos.csv.

In [ ]:
# Use Case - Time de Produto.
base_product = df[df['status'] == 'active'].copy()
base_product['gender'].value_counts()
base_product.to_csv("data/generos_ativos.csv", index=False)

gender
female    547
male      486
Name: count, dtype: int64

## **Conclusão** 

Este projeto implementou com sucesso um pipeline de dados ETL completo e funcional. Através da extração de dados da API GoRest, manipulação com Pandas e entrega de arquivos CSV direcionados.

O pipeline atendeu com sucesso a duas necessidades distintas: forneceu uma lista de prospecção para a equipe de Marketing reengajar usuários inativos e entregou um sumário demográfico para a equipe de Produto entender seu público ativo. Isso prova a capacidade de criar soluções de dados eficientes, seguras e automatizadas que capacitam diferentes áreas de uma empresa a tomar decisões mais inteligentes e ágeis.